In [1]:
import serial
import time
import threading
import struct
import string
from collections import deque
import numpy as np
from collections import Counter
from decimal import Decimal

RMID = 183          # RMID
TMID = 184          # TMID
ID = 1              # ID
PID_MAINBC = 10     # PID (MAINBC)
PID_CMD = 207       # PID (207 = RPM)
DATA_NUM = 7        # PID 207 DATA_NUM
WHEEL_L_ENABLE = 1  # L모터 Enable
WHEEL_R_ENABLE = 1  # R모터 Enable
DUMMY = 0           # CRC 전 DUMMY


In [2]:
def speed_msg(L_SPD, R_SPD):
    portM = "/dev/ttyMD400" #PC에 연결된 포트명
    baudM = 57600  # 시리얼 보드레이트(통신속도)
    serM = serial.Serial(portM, baudM, timeout=1)
    MOTOR_L_SPD = [L_SPD & 255,(L_SPD >> 8)&255]
    MOTOR_R_SPD = [R_SPD & 255,(R_SPD >> 8)&255]
    send_msg = [RMID, TMID, ID, PID_CMD, DATA_NUM, WHEEL_L_ENABLE, MOTOR_L_SPD[0],MOTOR_L_SPD[1],WHEEL_R_ENABLE,MOTOR_R_SPD[0],MOTOR_R_SPD[1],DUMMY]
    CHC=(~sum(send_msg)+1)&255
    send_msg.append(CHC)
    serM.write(send_msg)
    #print(L_SPD , 'SUCCESS')
    return

In [3]:
Lport = "/dev/ttyACM3"
Rport = "/dev/ttyACM4" #PC에 연결된 포트명
baud = 115200  # 시리얼 보드레이트(통신속도)
serL = serial.Serial(Lport, baud, timeout=1)
serR = serial.Serial(Rport, baud, timeout=1)

dist_L = deque([0,0,0,0,0])
dist_R = deque([0,0,0,0,0])
direction_Q = deque([0,0])
last_diff = 0
count = 0
last_direction = 0
while True:
    
    readL = 0
    readR = 0
    if serL.readable(): #값이 들어왔는지 확인          
        tempL = serL.readline().decode()
        if len(tempL) > 3:      
            resL = Decimal(str(tempL))
            readL = 1
    if serR.readable(): #값이 들어왔는지 확인   
        #print(serR.readline().decode())      
        tempR = serR.readline().decode()
        if len(tempR) > 3:
            resR = Decimal(str(tempR))
            calib = Decimal('0.03')
            resR = resR-calib
            readR = 1
        
    if readL == 1 and readR == 1:
        current_diff = abs(resL-resR)
        if abs(resL) < 50 and abs(resR) < 50 and abs(last_diff - current_diff) < 0.3:         
            dist_L.append(resL)
            dist_L.popleft()
            dist_R.append(resR)
            dist_R.popleft()
        else:
            print('filter')

        resL_avg = np.mean(dist_L)
        resR_avg = np.mean(dist_R)

        if abs(resL_avg-resR_avg) <= 0.05:
            #print(abs(resL_avg-resR_avg))
            direction = 'Center'
            if last_direction == 0:
                count = count+1
            else:
                count = 1
                last_direction = 0
            if count > 1:
                #speed_msg(0,0)
                if resL_avg > 2.0:
                     speed_msg(500,-500)
                else:  
                     speed_msg(0,0)
            #direction_Q.append('CENTER')
            #direction_Q.popleft()
        #elif resL_avg > resR_avg:
        elif (resL_avg - resR_avg) > 0.15:
            #print(resL_avg - resR_avg)
            direction = 'LEFT'
            if last_direction == 1:
                count = count+1
            else:
                count = 1
                last_direction = 1
            if count > 3:
                speed_msg(50,50)
                # print('LEFT_enable')
                # if resL_avg > 3.0:
                #     speed_msg(350,-250)
                # else:
                #     speed_msg(50,50)
            #direction_Q.append('LEFT')
            #direction_Q.popleft()
         #else:
        elif (resR_avg - resL_avg) > 0.15:
            #print(resR_avg - resL_avg)
            direction = 'RIGHT'
            if last_direction == 2:
                count = count+1
            else:
                count = 1
                last_direction = 2
            if count > 3:
                speed_msg(-50,-50)
                # print('RIGHT_ENABLE')
                # if resL_avg > 3.0:
                #     speed_msg(250,-350)
                # else:
                #     speed_msg(-50,-50)
        else:
            direction = 'NONE'
            if resL_avg < 2.0:
                speed_msg(0,0)
            #direction_Q.append('RIGHT')
            #direction_Q.popleft()
        
        #most = Counter(direction_Q).most_common(n=1)[0][0]
        #if most == 'CENTER':
        #    speed_msg(0,0)
        #elif most == "LEFT":
        #    speed_msg(-50,-50)
        #elif most == "RIGHT":
        #    speed_msg(50,50)
        
        last_diff = abs(resL-resR)
        print(resL, resR)
        print(resL_avg, resR_avg)
        print(resL_avg-resR_avg, count)
        #print(direction)
        #print(most)

stop_msg()
serR.close()
serL.close()

SerialException: [Errno 2] could not open port /dev/ttyACM3: [Errno 2] No such file or directory: '/dev/ttyACM3'

In [4]:
speed_msg(0,0)

In [3]:
speed_msg(100,100)

In [47]:
#v = 2 * π* r * w / 60
# V = 2 * math

import math

#전진
def forward_spd_ms(velocity):
    RPM = velocity / (2*3.14*0.01375) * 60
    
    
    
2*math.pi*0.01375*100/60*15

2.1598449493429825

In [41]:
velocity = 2
RPM = velocity/(2*math.pi*0.01375/60)
print(RPM*15)

20834.828913848116


In [60]:
(2*3.14*0.1375*60)*60

3108.6000000000004